<a href="https://colab.research.google.com/github/KanekoIrisu/1se-weather-app/blob/main/%E5%B0%B1%E6%B4%BB%E3%82%A2%E3%83%97%E3%83%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import customtkinter as ctk
import sqlite3
from tkinter import ttk, messagebox
import os
import datetime
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np

# Google Calendar API関連のインポート
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# --- Google Calendar API設定 ---
SCOPES = ['https://www.googleapis.com/auth/calendar']
# 色の定義 (Google Calendar API ColorId)
# https://developers.google.com/calendar/api/v3/reference/colors/get
COLOR_ID_ENTRY = '10' # 緑 (Basil)
COLOR_ID_EVENT = '7' # アクアマリン (Peacock)

# --- データベース設定 ---
DB_FILE = "shukatsu_app.db"

# --- レーダーチャート設定 ---
CHART_LABELS = ['外向性', '協調性', '好奇心', '勤勉性', '情緒安定性', '外見', '語学力', '専門性']


class ShukatsuApp(ctk.CTk):
    def __init__(self):
        super().__init__()

        self.title("就活管理アプリ")
        self.geometry("1400x850")
        ctk.set_appearance_mode("System") # or "Light", "Dark"
        ctk.set_default_color_theme("blue")

        self.db_conn = self.init_database()
        self.current_company_id = None
        self.chart_canvas = None

        self._create_widgets()
        self.load_company_list()
        self.load_selection_list()

    def init_database(self):
        """データベースを初期化し、テーブルを作成する"""
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        # 企業情報テーブル
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS companies (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL UNIQUE,
                extroversion REAL DEFAULT 0,
                agreeableness REAL DEFAULT 0,
                curiosity REAL DEFAULT 0,
                diligence REAL DEFAULT 0,
                stability REAL DEFAULT 0,
                appearance REAL DEFAULT 0,
                language REAL DEFAULT 0,
                specialty REAL DEFAULT 0,
                entry_deadline TEXT DEFAULT '',
                event_name TEXT DEFAULT '',
                event_datetime TEXT DEFAULT '',
                event_location TEXT DEFAULT '',
                interest_level TEXT DEFAULT '',
                website TEXT DEFAULT ''
            )
        ''')
        # 選考情報テーブル
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS selections (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                exam_date TEXT,
                exam_content TEXT,
                result TEXT
            )
        ''')
        conn.commit()
        return conn

    def _create_widgets(self):
        """メインウィンドウのウィジェットを作成・配置する"""
        self.tabview = ctk.CTkTabview(self, width=1380, height=830)
        self.tabview.pack(padx=10, pady=10, fill="both", expand=True)

        self.tab_company = self.tabview.add("企業管理")
        self.tab_selection = self.tabview.add("選考管理")
        self.tab_template = self.tabview.add("テンプレート")

        self._create_company_tab()
        self._create_selection_tab()
        self._create_template_tab()

    def _create_company_tab(self):
        """企業管理タブのUIを作成"""
        # --- レイアウトフレーム ---
        left_frame = ctk.CTkFrame(self.tab_company, width=400)
        left_frame.pack(side="left", fill="y", padx=10, pady=10)
        left_frame.pack_propagate(False)

        center_frame = ctk.CTkFrame(self.tab_company)
        center_frame.pack(side="left", fill="both", expand=True, padx=(0, 10), pady=10)

        right_frame = ctk.CTkFrame(self.tab_company, width=500)
        right_frame.pack(side="left", fill="y", padx=(0, 10), pady=10)
        right_frame.pack_propagate(False)

        # --- 左フレーム（入力フォーム） ---
        ctk.CTkLabel(left_frame, text="企業情報入力", font=ctk.CTkFont(size=18, weight="bold")).pack(pady=(10, 20))

        # 企業名
        ctk.CTkLabel(left_frame, text="企業名").pack(padx=20, pady=(10, 2), anchor="w")
        self.company_name_entry = ctk.CTkEntry(left_frame, placeholder_text="例: 株式会社〇〇")
        self.company_name_entry.pack(padx=20, pady=2, fill="x")

        # 企業履歴
        self.company_history_combo = ctk.CTkComboBox(left_frame, values=[], command=self.load_company_by_name)
        self.company_history_combo.pack(padx=20, pady=(5, 10), fill="x")
        self.company_history_combo.set("履歴から選択...")

        # イベント情報
        ctk.CTkLabel(left_frame, text="エントリー期限 (YYYY-MM-DD)").pack(padx=20, pady=(10, 2), anchor="w")
        self.entry_deadline_entry = ctk.CTkEntry(left_frame)
        self.entry_deadline_entry.pack(padx=20, pady=2, fill="x")

        ctk.CTkLabel(left_frame, text="イベント名").pack(padx=20, pady=(10, 2), anchor="w")
        self.event_name_entry = ctk.CTkEntry(left_frame)
        self.event_name_entry.pack(padx=20, pady=2, fill="x")

        ctk.CTkLabel(left_frame, text="開催日時 (YYYY-MM-DD HH:MM)").pack(padx=20, pady=(10, 2), anchor="w")
        self.event_datetime_entry = ctk.CTkEntry(left_frame)
        self.event_datetime_entry.pack(padx=20, pady=2, fill="x")

        ctk.CTkLabel(left_frame, text="開催場所").pack(padx=20, pady=(10, 2), anchor="w")
        self.event_location_entry = ctk.CTkEntry(left_frame)
        self.event_location_entry.pack(padx=20, pady=2, fill="x")

        ctk.CTkLabel(left_frame, text="やりたさ (0-5)").pack(padx=20, pady=(10, 2), anchor="w")
        self.interest_level_entry = ctk.CTkEntry(left_frame)
        self.interest_level_entry.pack(padx=20, pady=2, fill="x")

        ctk.CTkLabel(left_frame, text="就活サイトURL").pack(padx=20, pady=(10, 2), anchor="w")
        self.website_entry = ctk.CTkEntry(left_frame)
        self.website_entry.pack(padx=20, pady=2, fill="x")

        # 操作ボタン
        button_frame = ctk.CTkFrame(left_frame, fg_color="transparent")
        button_frame.pack(pady=20, fill="x")

        self.save_button = ctk.CTkButton(button_frame, text="保存 / 更新", command=self.save_company_data)
        self.save_button.pack(side="left", expand=True, padx=10)

        self.new_button = ctk.CTkButton(button_frame, text="新規作成", command=self.clear_company_form)
        self.new_button.pack(side="left", expand=True, padx=10)

        self.delete_button = ctk.CTkButton(button_frame, text="削除", command=self.delete_company_data, fg_color="#D32F2F", hover_color="#B71C1C")
        self.delete_button.pack(pady=(10,0), padx=20, fill="x")

        self.calendar_button = ctk.CTkButton(left_frame, text="Googleカレンダーに登録", command=self.add_to_google_calendar)
        self.calendar_button.pack(pady=10, padx=20, fill="x")

        # --- 中央フレーム（レーダーチャートと評価スライダー） ---
        chart_frame = ctk.CTkFrame(center_frame)
        chart_frame.pack(pady=10, padx=10, fill="both", expand=True)

        self.fig = Figure(figsize=(5, 5), dpi=100)
        self.ax = self.fig.add_subplot(111, polar=True)
        self.chart_canvas = FigureCanvasTkAgg(self.fig, master=chart_frame)
        self.chart_canvas.get_tk_widget().pack(side="top", fill="both", expand=True)

        self.sliders = {}
        slider_frame = ctk.CTkScrollableFrame(center_frame, label_text="性格・能力評価 (0-5)", height=250)
        slider_frame.pack(pady=10, padx=10, fill="x")

        for i, label in enumerate(CHART_LABELS):
            frame = ctk.CTkFrame(slider_frame)
            frame.pack(fill="x", padx=5, pady=5)

            lab = ctk.CTkLabel(frame, text=label, width=80)
            lab.pack(side="left", padx=5)

            slider_val_label = ctk.CTkLabel(frame, text="0.0", width=30)
            slider_val_label.pack(side="right", padx=5)

            slider = ctk.CTkSlider(frame, from_=0, to=5, number_of_steps=50, command=lambda value, l=slider_val_label, s=label: self.update_slider_value(value, l, s))
            slider.pack(side="left", fill="x", expand=True)
            slider.set(0)

            self.sliders[label] = (slider, slider_val_label)

        self.update_radar_chart()

        # --- 右フレーム（企業一覧） ---
        ctk.CTkLabel(right_frame, text="企業一覧", font=ctk.CTkFont(size=18, weight="bold")).pack(pady=10)

        cols = ('企業名', '性格(AVG)', 'エントリー期限', 'イベント名', '開催日時', '開催場所', 'やりたさ', '就活サイト')
        self.company_tree = ttk.Treeview(right_frame, columns=cols, show='headings', selectmode='browse')

        for col in cols:
            self.company_tree.heading(col, text=col)
            self.company_tree.column(col, width=100, anchor='w')
        self.company_tree.column('企業名', width=120)
        self.company_tree.column('性格(AVG)', width=60, anchor='center')

        self.company_tree.pack(fill="both", expand=True, padx=10, pady=5)
        self.company_tree.bind('<<TreeviewSelect>>', self.on_company_select)

    def _create_selection_tab(self):
        """選考管理タブのUIを作成"""
        top_frame = ctk.CTkFrame(self.tab_selection, height=150)
        top_frame.pack(fill="x", padx=10, pady=10)
        top_frame.grid_propagate(False)

        ctk.CTkLabel(top_frame, text="選考情報入力", font=ctk.CTkFont(size=16, weight="bold")).grid(row=0, column=0, columnspan=3, pady=10)

        ctk.CTkLabel(top_frame, text="試験日 (YYYY-MM-DD)").grid(row=1, column=0, padx=10, pady=5, sticky="w")
        self.exam_date_entry = ctk.CTkEntry(top_frame)
        self.exam_date_entry.grid(row=2, column=0, padx=10, pady=5, sticky="ew")

        ctk.CTkLabel(top_frame, text="試験内容 (SPI, 面接など)").grid(row=1, column=1, padx=10, pady=5, sticky="w")
        self.exam_content_entry = ctk.CTkEntry(top_frame)
        self.exam_content_entry.grid(row=2, column=1, padx=10, pady=5, sticky="ew")

        ctk.CTkLabel(top_frame, text="結果").grid(row=1, column=2, padx=10, pady=5, sticky="w")
        self.exam_result_combo = ctk.CTkComboBox(top_frame, values=["未定", "通過", "不合格", "辞退"])
        self.exam_result_combo.grid(row=2, column=2, padx=10, pady=5, sticky="ew")

        add_button = ctk.CTkButton(top_frame, text="追加", command=self.add_selection_data)
        add_button.grid(row=2, column=3, padx=20, pady=5)

        top_frame.grid_columnconfigure(0, weight=1)
        top_frame.grid_columnconfigure(1, weight=2)
        top_frame.grid_columnconfigure(2, weight=1)

        # 選考一覧
        bottom_frame = ctk.CTkFrame(self.tab_selection)
        bottom_frame.pack(fill="both", expand=True, padx=10, pady=(0, 10))

        cols = ('試験日', '試験内容', '結果')
        self.selection_tree = ttk.Treeview(bottom_frame, columns=cols, show='headings')
        for col in cols:
            self.selection_tree.heading(col, text=col)
        self.selection_tree.pack(fill="both", expand=True, padx=10, pady=10)

        delete_button_selection = ctk.CTkButton(bottom_frame, text="選択した項目を削除", command=self.delete_selection_data, fg_color="#D32F2F", hover_color="#B71C1C")
        delete_button_selection.pack(pady=5)

    def _create_template_tab(self):
        """テンプレートタブのUIを作成"""
        ctk.CTkLabel(self.tab_template, text="テンプレート", font=ctk.CTkFont(size=18, weight="bold")).pack(pady=10)

        self.template_textbox = ctk.CTkTextbox(self.tab_template, font=ctk.CTkFont(size=14))
        self.template_textbox.pack(fill="both", expand=True, padx=20, pady=10)

        # サンプルテンプレートを挿入
        template_text = """【自己PRテンプレート】
私の強みは〇〇です。この強みは、大学時代の△△の経験で培われました。
（具体的なエピソード）
この強みを活かし、貴社では□□の分野で貢献したいと考えております。

---

【ガクチカテンプレート】
私が学生時代に最も力を入れたことは、〇〇サークルでの活動です。
（目標、課題、行動、結果を具体的に）
この経験から、△△という学びを得ました。
"""
        self.template_textbox.insert("1.0", template_text)

    # --- ロジック関数 ---

    def update_slider_value(self, value, label_widget, slider_name):
        """スライダーの値を更新し、チャートを再描画する"""
        label_widget.configure(text=f"{value:.1f}")
        self.update_radar_chart()

    def update_radar_chart(self):
        """現在のスライダーの値でレーダーチャートを更新する"""
        values = [self.sliders[label][0].get() for label in CHART_LABELS]
        self.ax.clear()

        angles = np.linspace(0, 2 * np.pi, len(CHART_LABELS), endpoint=False).tolist()
        values += values[:1]
        angles += angles[:1]

        self.ax.fill(angles, values, color='red', alpha=0.25)
        self.ax.plot(angles, values, color='red', linewidth=2)

        self.ax.set_yticklabels([])
        self.ax.set_xticks(angles[:-1])
        self.ax.set_xticklabels(CHART_LABELS, fontfamily='Yu Gothic', fontsize=10) # Windowsの場合
        # self.ax.set_xticklabels(CHART_LABELS, fontfamily='Hiragino Sans', fontsize=10) # Macの場合

        self.ax.set_rlim(0, 5) # 評価の範囲を0-5に設定
        for i in range(1, 6):
            self.ax.text(0, i, str(i), fontsize=8, color='gray', ha='center')

        self.fig.canvas.draw()

    def load_company_list(self):
        """DBから企業一覧を読み込み、TreeviewとComboBoxを更新"""
        for i in self.company_tree.get_children():
            self.company_tree.delete(i)

        cursor = self.db_conn.cursor()
        cursor.execute("SELECT id, name, extroversion, agreeableness, curiosity, diligence, stability, appearance, language, specialty, entry_deadline, event_name, event_datetime, event_location, interest_level, website FROM companies ORDER BY name")
        rows = cursor.fetchall()

        company_names = []
        for row in rows:
            company_id, name, *ratings, deadline, event_name, event_dt, location, interest, site = row
            company_names.append(name)

            # 性格評価の平均値を計算
            avg_rating = sum(ratings) / len(ratings) if ratings else 0

            self.company_tree.insert('', 'end', iid=company_id, values=(
                name, f"{avg_rating:.1f}", deadline, event_name, event_dt, location, interest, site
            ))

        self.company_history_combo.configure(values=company_names)
        if not company_names:
            self.company_history_combo.set("履歴がありません")
        else:
            self.company_history_combo.set("履歴から選択...")

    def on_company_select(self, event):
        """Treeviewで企業が選択されたときの処理"""
        selected_items = self.company_tree.selection()
        if not selected_items:
            return

        self.current_company_id = selected_items[0]
        cursor = self.db_conn.cursor()
        cursor.execute("SELECT * FROM companies WHERE id = ?", (self.current_company_id,))
        data = cursor.fetchone()

        if data:
            self.fill_form_with_data(data)

    def load_company_by_name(self, company_name):
        """ComboBoxで企業名が選択されたときの処理"""
        if company_name == "履歴から選択...":
            return
        cursor = self.db_conn.cursor()
        cursor.execute("SELECT * FROM companies WHERE name = ?", (company_name,))
        data = cursor.fetchone()

        if data:
            self.current_company_id = data[0]
            self.fill_form_with_data(data)

    def fill_form_with_data(self, data):
        """フォームにデータを入力する"""
        (id, name, extro, agree, curio, dilig, stabil, appear, lang, spec,
         deadline, event_name, event_dt, location, interest, site) = data

        self.company_name_entry.delete(0, 'end'); self.company_name_entry.insert(0, name)
        self.entry_deadline_entry.delete(0, 'end'); self.entry_deadline_entry.insert(0, deadline)
        self.event_name_entry.delete(0, 'end'); self.event_name_entry.insert(0, event_name)
        self.event_datetime_entry.delete(0, 'end'); self.event_datetime_entry.insert(0, event_dt)
        self.event_location_entry.delete(0, 'end'); self.event_location_entry.insert(0, location)
        self.interest_level_entry.delete(0, 'end'); self.interest_level_entry.insert(0, interest)
        self.website_entry.delete(0, 'end'); self.website_entry.insert(0, site)

        ratings = {
            '外向性': extro, '協調性': agree, '好奇心': curio, '勤勉性': dilig,
            '情緒安定性': stabil, '外見': appear, '語学力': lang, '専門性': spec
        }

        for label, value in ratings.items():
            slider, val_label = self.sliders[label]
            slider.set(value)
            val_label.configure(text=f"{value:.1f}")

        self.update_radar_chart()

    def clear_company_form(self):
        """企業情報フォームをクリアする"""
        self.current_company_id = None
        self.company_name_entry.delete(0, 'end')
        self.entry_deadline_entry.delete(0, 'end')
        self.event_name_entry.delete(0, 'end')
        self.event_datetime_entry.delete(0, 'end')
        self.event_location_entry.delete(0, 'end')
        self.interest_level_entry.delete(0, 'end')
        self.website_entry.delete(0, 'end')

        for label in CHART_LABELS:
            slider, val_label = self.sliders[label]
            slider.set(0)
            val_label.configure(text="0.0")

        self.update_radar_chart()
        self.company_tree.selection_remove(self.company_tree.selection())
        self.company_history_combo.set("履歴から選択...")
        messagebox.showinfo("情報", "新規作成モードになりました。")

    def save_company_data(self):
        """企業データを保存または更新する"""
        name = self.company_name_entry.get().strip()
        if not name:
            messagebox.showerror("エラー", "企業名は必須です。")
            return

        data = {
            'name': name,
            'extroversion': self.sliders['外向性'][0].get(),
            'agreeableness': self.sliders['協調性'][0].get(),
            'curiosity': self.sliders['好奇心'][0].get(),
            'diligence': self.sliders['勤勉性'][0].get(),
            'stability': self.sliders['情緒安定性'][0].get(),
            'appearance': self.sliders['外見'][0].get(),
            'language': self.sliders['語学力'][0].get(),
            'specialty': self.sliders['専門性'][0].get(),
            'entry_deadline': self.entry_deadline_entry.get(),
            'event_name': self.event_name_entry.get(),
            'event_datetime': self.event_datetime_entry.get(),
            'event_location': self.event_location_entry.get(),
            'interest_level': self.interest_level_entry.get(),
            'website': self.website_entry.get(),
        }

        cursor = self.db_conn.cursor()
        try:
            if self.current_company_id: # 更新
                # idをdataに追加
                data['id'] = self.current_company_id
                cursor.execute('''
                    UPDATE companies SET
                    name=:name, extroversion=:extroversion, agreeableness=:agreeableness, curiosity=:curiosity,
                    diligence=:diligence, stability=:stability, appearance=:appearance, language=:language,
                    specialty=:specialty, entry_deadline=:entry_deadline, event_name=:event_name,
                    event_datetime=:event_datetime, event_location=:event_location, interest_level=:interest_level,
                    website=:website WHERE id=:id
                ''', data)
            else: # 新規作成
                cursor.execute('''
                    INSERT INTO companies (
                        name, extroversion, agreeableness, curiosity, diligence, stability, appearance,
                        language, specialty, entry_deadline, event_name, event_datetime, event_location,
                        interest_level, website
                    ) VALUES (
                        :name, :extroversion, :agreeableness, :curiosity, :diligence, :stability, :appearance,
                        :language, :specialty, :entry_deadline, :event_name, :event_datetime, :event_location,
                        :interest_level, :website
                    )
                ''', data)
            self.db_conn.commit()
            messagebox.showinfo("成功", f"企業「{name}」の情報を保存しました。")
            self.load_company_list()
            self.clear_company_form()

        except sqlite3.IntegrityError:
            messagebox.showerror("エラー", "この企業名は既に存在します。")
        except Exception as e:
            messagebox.showerror("データベースエラー", f"エラーが発生しました: {e}")

    def delete_company_data(self):
        """選択中の企業データを削除する"""
        if not self.current_company_id:
            messagebox.showwarning("警告", "削除する企業を選択してください。")
            return

        company_name = self.company_name_entry.get()
        if messagebox.askyesno("確認", f"本当に企業「{company_name}」を削除しますか？"):
            cursor = self.db_conn.cursor()
            cursor.execute("DELETE FROM companies WHERE id = ?", (self.current_company_id,))
            self.db_conn.commit()
            messagebox.showinfo("成功", f"企業「{company_name}」を削除しました。")
            self.load_company_list()
            self.clear_company_form()

    # --- 選考管理ロジック ---
    def add_selection_data(self):
        """選考情報をDBに追加"""
        date = self.exam_date_entry.get()
        content = self.exam_content_entry.get()
        result = self.exam_result_combo.get()

        if not date or not content:
            messagebox.showerror("エラー", "試験日と試験内容は必須です。")
            return

        cursor = self.db_conn.cursor()
        cursor.execute("INSERT INTO selections (exam_date, exam_content, result) VALUES (?, ?, ?)",
                       (date, content, result))
        self.db_conn.commit()
        self.load_selection_list()

        # 入力欄をクリア
        self.exam_date_entry.delete(0, 'end')
        self.exam_content_entry.delete(0, 'end')
        self.exam_result_combo.set("未定")

    def load_selection_list(self):
        """選考一覧をTreeviewに読み込む"""
        for i in self.selection_tree.get_children():
            self.selection_tree.delete(i)

        cursor = self.db_conn.cursor()
        cursor.execute("SELECT id, exam_date, exam_content, result FROM selections ORDER BY exam_date DESC")
        rows = cursor.fetchall()

        for row in rows:
            self.selection_tree.insert('', 'end', iid=row[0], values=(row[1], row[2], row[3]))

    def delete_selection_data(self):
        """選択した選考情報を削除"""
        selected_items = self.selection_tree.selection()
        if not selected_items:
            messagebox.showwarning("警告", "削除する項目を選択してください。")
            return

        if messagebox.askyesno("確認", "選択した項目を本当に削除しますか？"):
            cursor = self.db_conn.cursor()
            for item_id in selected_items:
                cursor.execute("DELETE FROM selections WHERE id = ?", (item_id,))
            self.db_conn.commit()
            self.load_selection_list()

    # --- Google Calendar 連携 ---
    def get_calendar_service(self):
        """Google Calendar APIのサービスオブジェクトを取得する"""
        creds = None
        if os.path.exists('token.json'):
            creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                if not os.path.exists('credentials.json'):
                    messagebox.showerror("エラー", "credentials.json が見つかりません。\nステップ2の手順を再度確認してください。")
                    return None
                flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)
            with open('token.json', 'w') as token:
                token.write(creds.to_json())

        try:
            service = build('calendar', 'v3', credentials=creds)
            return service
        except HttpError as error:
            messagebox.showerror("APIエラー", f'An error occurred: {error}')
            return None

    def add_to_google_calendar(self):
        """現在の企業の予定をGoogleカレンダーに登録する"""
        if not self.current_company_id:
            messagebox.showwarning("警告", "カレンダーに登録する企業を選択してください。")
            return

        company_name = self.company_name_entry.get()
        entry_deadline_str = self.entry_deadline_entry.get()
        event_datetime_str = self.event_datetime_entry.get()

        if not entry_deadline_str and not event_datetime_str:
            messagebox.showinfo("情報", "登録する日付が入力されていません。")
            return

        service = self.get_calendar_service()
        if not service:
            return

        # 1. エントリー期限の登録
        if entry_deadline_str:
            try:
                deadline_date = datetime.datetime.strptime(entry_deadline_str, '%Y-%m-%d').date()
                event = {
                    'summary': f'[ES締切] {company_name}',
                    'description': f'{company_name} のエントリーシート提出期限です。',
                    'start': {'date': deadline_date.isoformat()},
                    'end': {'date': (deadline_date + datetime.timedelta(days=1)).isoformat()}, # 終日イベント
                    'colorId': COLOR_ID_ENTRY,
                }
                service.events().insert(calendarId='primary', body=event).execute()
                messagebox.showinfo("成功", f"「{company_name}」のエントリー期限をカレンダーに登録しました。")
            except ValueError:
                messagebox.showerror("エラー", "エントリー期限の形式が不正です (YYYY-MM-DD)。")
            except HttpError as e:
                messagebox.showerror("カレンダーAPIエラー", f"イベント作成中にエラーが発生しました: {e}")

        # 2. 開催日時の登録
        if event_datetime_str:
            try:
                start_time = datetime.datetime.strptime(event_datetime_str, '%Y-%m-%d %H:%M')
                # 終了時間は開始の1時間後と仮定
                end_time = start_time + datetime.timedelta(hours=1)
                event = {
                    'summary': f'[{self.event_name_entry.get()}] {company_name}',
                    'location': self.event_location_entry.get(),
                    'description': f'{company_name} の {self.event_name_entry.get()} が開催されます。',
                    'start': {'dateTime': start_time.isoformat(), 'timeZone': 'Asia/Tokyo'},
                    'end': {'dateTime': end_time.isoformat(), 'timeZone': 'Asia/Tokyo'},
                    'colorId': COLOR_ID_EVENT,
                }
                service.events().insert(calendarId='primary', body=event).execute()
                messagebox.showinfo("成功", f"「{company_name}」の開催日時をカレンダーに登録しました。")
            except ValueError:
                messagebox.showerror("エラー", "開催日時の形式が不正です (YYYY-MM-DD HH:MM)。")
            except HttpError as e:
                messagebox.showerror("カレンダーAPIエラー", f"イベント作成中にエラーが発生しました: {e}")

    def on_closing(self):
        """アプリ終了時にDB接続を閉じる"""
        if self.db_conn:
            self.db_conn.close()
        self.destroy()

if __name__ == "__main__":
    # 日本語フォントがない場合のエラーを避けるため、フォントの存在を確認
    try:
        # Windows/Macで一般的に利用可能な日本語フォントを指定
        from matplotlib import font_manager
        fonts = [f.name for f in font_manager.fontManager.ttflist]
        if 'Yu Gothic' not in fonts and 'Hiragino Sans' not in fonts:
            print("警告: Yu GothicまたはHiragino Sansフォントが見つかりません。チャートの日本語が文字化けする可能性があります。")
    except Exception as e:
        print(f"フォントチェック中にエラー: {e}")

    app = ShukatsuApp()
    app.protocol("WM_DELETE_WINDOW", app.on_closing)
    app.mainloop()